# 🎓 Smart Study Buddy - Colab Notebook

An adaptive AI tutor that explains any topic perfectly matched to your audience.

---

## Setup

In [ ]:
# Install required packages
!pip install -q openai anthropic python-dotenv rich

In [ ]:
# Clone repository (if running from GitHub)
# !git clone https://github.com/yourusername/smart-study-buddy.git
# %cd smart-study-buddy

## 🔑 Configure API Keys

Add your API keys below:

In [ ]:
import os

# Set your API keys
os.environ["OPENAI_API_KEY"] = "sk-proj-ullXBO88G92FU49y0H7MgXgf0qSHnRB7zNayJWCM9SU5A8VsgQEZBxKZ8LrKTBM_MUPwDhNzGIT3BlbkFJ7AOBl0VPPS"

# Optional: Anthropic API key if you want to use Claude
# os.environ["ANTHROPIC_API_KEY"] = "your_key_here"

# Optional: Pinecone for future features
# os.environ["PINECONE_API_KEY"] = "pcsk_7PupGf_8WtvCTW8EQVUR3X6nFEQWJW6x6WNwdFNbkMrgTwraXVZwCBcKvghUthsE9TCSUY"

print("✅ API keys configured!")

## 📚 Import Smart Study Buddy

In [ ]:
# If you cloned the repo, use this:
# from src.study_buddy import SmartStudyBuddy, quick_explain, explain_for_child, explain_for_expert

# Otherwise, copy the code directly (see next cell)

In [ ]:
# STANDALONE VERSION - Copy all necessary code here

from typing import Optional
from openai import OpenAI

# System Prompt
SYSTEM_PROMPT = """You are Smart Study Buddy, an adaptive AI tutor designed to explain any topic in a way that perfectly matches the learner's age, level, and background.

Your goal is clarity first, confidence always. You make complex ideas feel simple, friendly, and approachable—without losing accuracy.

## How You Receive Input

You will always receive:
* Topic: the subject to explain
* Audience: age or experience level (e.g., 5-year-old, middle school, beginner, advanced, expert)

Optional inputs may include:
* Tone: playful, neutral, academic, professional
* Length: short, medium, detailed

## How You Must Adapt Your Explanation

**For young learners / beginners:**
* Use simple words
* Short sentences
* Friendly, encouraging tone
* Everyday examples and metaphors
* No jargon unless explained gently

**For intermediate learners:**
* Clear definitions
* Real-world examples
* Light technical terms with explanations
* Logical flow

**For advanced / expert learners:**
* Precise terminology
* Deeper explanations
* Formulas, mechanisms, or theories when relevant
* Minimal simplification

## Teaching Structure (Mandatory)

Always follow this structure:
1. Simple core idea (one or two sentences)
2. Explanation adapted to the audience
3. Example or analogy
4. Optional deeper insight (only if appropriate for the audience)

## Style Rules

* Never sound condescending
* Never assume prior knowledge unless the audience is expert
* Keep explanations engaging and motivating
* Avoid unnecessary complexity
* Prefer clarity over verbosity"""

# Audience levels
AUDIENCE_LEVELS = {
    "child": "5-year-old child",
    "elementary": "elementary school student (ages 6-10)",
    "middle_school": "middle school student (ages 11-14)",
    "high_school": "high school student (ages 15-18)",
    "beginner": "beginner adult with no prior knowledge",
    "intermediate": "intermediate learner with basic knowledge",
    "advanced": "advanced learner with substantial background",
    "expert": "expert in the field"
}

class SmartStudyBuddy:
    def __init__(self, api_key=None):
        self.client = OpenAI(api_key=api_key or os.environ.get("OPENAI_API_KEY"))
    
    def explain(self, topic: str, audience: str = "beginner", 
                tone: str = None, length: str = None):
        # Resolve audience shorthand
        audience = AUDIENCE_LEVELS.get(audience, audience)
        
        # Build prompt
        prompt_parts = [f"Topic: {topic}", f"Audience: {audience}"]
        if tone:
            prompt_parts.append(f"Tone: {tone}")
        if length:
            prompt_parts.append(f"Length: {length}")
        prompt_parts.append("\nPlease explain this topic according to the guidelines above.")
        
        user_prompt = "\n".join(prompt_parts)
        
        # Call API
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=2000,
            temperature=0.7
        )
        
        return response.choices[0].message.content

# Quick functions
def quick_explain(topic, audience="beginner"):
    buddy = SmartStudyBuddy()
    return buddy.explain(topic, audience)

def explain_for_child(topic):
    return quick_explain(topic, "child")

def explain_for_expert(topic):
    return quick_explain(topic, "expert")

print("✅ Smart Study Buddy loaded!")

## 🚀 Quick Start Examples

In [ ]:
# Example 1: Explain quantum physics to a 5-year-old
print(explain_for_child("quantum physics"))

In [ ]:
# Example 2: Explain photosynthesis to a middle schooler
print(quick_explain("photosynthesis", "middle_school"))

In [ ]:
# Example 3: Explain neural networks to an expert
print(explain_for_expert("neural networks"))

## 🎯 Advanced Usage

In [ ]:
# Initialize Smart Study Buddy
buddy = SmartStudyBuddy()

# Custom explanation with all parameters
explanation = buddy.explain(
    topic="blockchain technology",
    audience="intermediate",
    tone="professional",
    length="detailed"
)

print(explanation)

In [ ]:
# Compare explanations for different audiences
topic = "how computers work"

print("=" * 60)
print("FOR A CHILD:")
print("=" * 60)
print(buddy.explain(topic, "child"))
print("\n" + "=" * 60)
print("FOR AN EXPERT:")
print("=" * 60)
print(buddy.explain(topic, "expert"))

## 🧪 Interactive Widget

In [ ]:
# Install ipywidgets if needed
!pip install -q ipywidgets

import ipywidgets as widgets
from IPython.display import display, Markdown

# Create widgets
topic_widget = widgets.Text(
    description='Topic:',
    placeholder='Enter a topic...'
)

audience_widget = widgets.Dropdown(
    options=list(AUDIENCE_LEVELS.keys()),
    description='Audience:',
    value='beginner'
)

tone_widget = widgets.Dropdown(
    options=['None', 'playful', 'neutral', 'academic', 'professional'],
    description='Tone:',
    value='None'
)

length_widget = widgets.Dropdown(
    options=['None', 'short', 'medium', 'detailed'],
    description='Length:',
    value='None'
)

button = widgets.Button(
    description='✨ Explain',
    button_style='success'
)

output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        print("🔄 Generating explanation...")
        
        tone = None if tone_widget.value == 'None' else tone_widget.value
        length = None if length_widget.value == 'None' else length_widget.value
        
        explanation = buddy.explain(
            topic_widget.value,
            audience_widget.value,
            tone,
            length
        )
        
        output.clear_output()
        display(Markdown(f"### Explanation\n\n{explanation}"))

button.on_click(on_button_click)

# Display interface
display(widgets.VBox([
    widgets.HTML("<h2>🎓 Smart Study Buddy</h2>"),
    topic_widget,
    audience_widget,
    tone_widget,
    length_widget,
    button,
    output
]))

## 📊 Batch Processing

In [ ]:
# Explain multiple topics for the same audience
topics = [
    "gravity",
    "photosynthesis",
    "the water cycle",
    "electricity"
]

audience = "elementary"

for topic in topics:
    print(f"\n{'=' * 60}")
    print(f"📚 {topic.upper()}")
    print(f"{'=' * 60}\n")
    print(buddy.explain(topic, audience))
    print()

## 💾 Save Explanations

In [ ]:
# Save an explanation to a file
topic = "quantum entanglement"
audience = "high_school"

explanation = buddy.explain(topic, audience)

filename = f"{topic.replace(' ', '_')}_{audience}.txt"
with open(filename, 'w') as f:
    f.write(f"Topic: {topic}\n")
    f.write(f"Audience: {audience}\n")
    f.write("=" * 60 + "\n\n")
    f.write(explanation)

print(f"✅ Saved to {filename}")

## 🎨 Experiment Zone

Try your own topics and audiences below:

In [ ]:
# Your experiments here!
my_topic = "___"
my_audience = "___"

print(buddy.explain(my_topic, my_audience))